In [ ]:
import os
import torch
from pathlib import Path

## General

### Parallel computing

In [ ]:
if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
    print(int(os.environ["RANK"]))
    print(int(os.environ['WORLD_SIZE']))
    print(int(os.environ['LOCAL_RANK']))
elif 'SLURM_PROCID' in os.environ:
    print(int(os.environ['SLURM_PROCID']))
    print( torch.cuda.device_count())
else:
    print('Not using distributed mode')

Parallel computing is a minor issue for now that needs to be solved eventually, Pytorch lightning integration might be useful.

### Dataset

Datasets seem to keeo COCO annotation format with a json file with an "annotations" key and corresponding value "0" for lines. 

Question: difference between id and category id in coco:

    - categories are classes (or labels) of objects that are present in an image.
    - category_id : This maps the category that an object belongs to.
    - image_id : This is the mapping with the image that the object is related to. 
    - id : This is a unique identifier that identifies each annotation.




Let's load some images and look at ground truth annotations. 

In [ ]:
DATADIR = Path("../data")

In [ ]:
root = DATADIR / "wireframe_processed"
mode = 'lines'

In [ ]:
PATHS = {
            "train": (root / "train2017", root / "annotations" / f'{mode}_train2017.json'),
            "val": (root / "val2017", root / "annotations" / f'{mode}_val2017.json'),
        }    

In [ ]:
PATHS

Let us first look at the train set

In [ ]:
image_set = "train"

In [ ]:
img_folder, ann_file = PATHS[image_set]

In [ ]:
from datasets.coco import ConvertCocoPolysToMask, make_coco_transforms

TODO: fix path issue when loading local module.  

In [ ]:
import torch
import torch.utils.data
import torchvision

import datasets.transforms as T
import math
import numpy as np

In [ ]:

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, ann_file, transforms, args, quit=False):
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self._transforms = transforms
        self.prepare = ConvertCocoPolysToMask() 
        self.args = args
        self.quit = quit
    def __getitem__(self, idx):
        img, target = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        img, target = self.prepare(img, target, self.args)
        if self.quit:
            return img, target
         
        if self._transforms is not None:
            print("transforming img")
            img, new_target = self._transforms(img, target)
        return img, new_target


In [ ]:
class MyArgs:
    def __init__(self, dictionary):
        self.__dict__ = dictionary
    
    def __getattr__(self, key):
        return self.__dict__[key]
my_dict = {"eval": False, "coco_path": "../data/wireframe_processed"}
args = MyArgs(my_dict)
print(args.eval) # output: John

In [ ]:

from datasets import build_dataset
train_dataset = build_dataset(image_set='train', args=args)

In [ ]:
prepare_dataset = CocoDetection(img_folder, ann_file, transforms=make_coco_transforms(image_set, args), args=args, quit=True)

In [ ]:
i=1
img = prepare_dataset[i][0]
img_lines = prepare_dataset[i][1]["lines"]

In [ ]:
prepare_dataset[0][1].keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(img_lines.max())
print(img_lines.min())

In [ ]:
prepare_dataset[0][0].size

In [ ]:
img = prepare_dataset[i][0]

In [ ]:
fig = plt.figure()
plt.imshow(img)
for tp_id, line in enumerate(img_lines):
    x1,y1, x2, y2 = line.numpy() # this is xyxy # TODO: check why the output of the network is inverted
    p1 = (x1, y1)
    p2 = (x2, y2)
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], linewidth=1, color='red', zorder=1)
# plt.axis('off')

#### Flipping the image

In [ ]:
import torchvision.transforms.functional as F


In [ ]:
flipped_image = F.hflip(img)

w, h = img.size


lines = img_lines.clone()
lines = lines[:, [2, 3, 0, 1]] * torch.as_tensor([-1, 1, -1, 1]) + torch.as_tensor([w, 0, w, 0])


In [ ]:
fig = plt.figure()
plt.imshow(flipped_image)
for tp_id, line in enumerate(lines):
    x1,y1, x2, y2 = line.numpy() # this is xyxy # TODO: check why the output of the network is inverted
    p1 = (x1, y1)
    p2 = (x2, y2)
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], linewidth=1, color='red', zorder=1)
# plt.axis('off')

In [ ]:
dataset = CocoDetection(img_folder, ann_file, transforms=make_coco_transforms(image_set, args), args=args)


In [ ]:
dataset[0][1].keys()

In [ ]:
img = train_dataset[i][0]
# img_lines_new = dataset[0][1]["lines"]
target =  train_dataset[i][1]

In [ ]:
target["size"]

In [ ]:
target["orig_size"]

In [ ]:
target["lines"]

In [ ]:
img_h, img_w = target["orig_size"].unbind(0)
scale_fct = torch.unsqueeze(torch.stack([img_w, img_h, img_w, img_h], dim=0), dim=0)
lines = target["lines"] * scale_fct[:, None, :]
# lines = lines.view(1000, 2, 2)
# lines = lines.flip([-1])# this is yxyx format

In [ ]:
lines.max()

In [ ]:
lines.shape

In [ ]:
import datasets.transforms as T


In [ ]:
target["lines"].numpy()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for e in lines:
    break

In [ ]:
lines.shape

In [ ]:
def plot_lines(lines, ax):
    for tp_id, line in enumerate(lines.squeeze()):
        x1,y1, x2, y2 = line.numpy() # this is xyxy # TODO: check why the output of the network is inverted
        p1 = (x1, y1)
        p2 = (x2, y2)
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], linewidth=1, color='red', zorder=1)
fig, axes = plt.subplots(1, 2, figsize = (20, 20))
axes[0].imshow(img.permute(1,2,0))
plot_lines(lines, axes[1])
plt.show()

In [ ]:
fig = plt.figure()
plt.imshow(img.permute(1,2,0))
plt.axis('off')
fig = plt.figure()
for tp_id, line in enumerate(target["lines"].squeeze()):
    x1,y1, x2, y2 = line.numpy() # this is xyxy # TODO: check why the output of the network is inverted
    # y1, x1, y2, x2 = line.numpy() # this is yxyx
    p1 = (x1 , y1 )
    p2 = (x2 , y2 )
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], linewidth=1, color='red', zorder=1)
plt.axis('off')